In [2]:
import pandas as pd
import os
from utils import *

In [3]:
baci = pd.read_csv(f"{data_paths['baci']}/BACI_HS12_Y2012_V202501.csv")
trustworthiness_scores = pd.read_csv(f"{data_paths['graphs_data']}/trustworthiness_scores.csv", dtype={"cmd": str})
atlas_df = pd.concat([pd.read_stata(f"{data_paths['atlas']}/hs12_country_country_product_year_4_2012_2016.dta"),
                pd.read_stata(f"{data_paths['atlas']}/hs12_country_country_product_year_4_2017_2021.dta"),
                pd.read_stata(f"{data_paths['atlas']}/hs12_country_country_product_year_4_2022.dta")])
atlas_countries = pd.read_csv(f"{data_paths['atlas']}/location_country.csv", encoding="latin")
reporters = pd.read_csv(f"{data_paths['uncomtrade']}/reporters.csv")

Codes in UN Comtrade not matching Atlas

In [4]:
trustworthiness_scores[~trustworthiness_scores.country_id.isin(atlas_countries.country_id)]["country_id"].unique()

array([886, 975,  58, 866, 588, 230,  97, 280, 251, 699, 532, 579, 490,
       590, 459, 717, 461, 658, 711, 736, 757, 835, 842, 841, 836],
      dtype=int64)

In [5]:
trustworthiness_scores.loc[trustworthiness_scores.country_id == 251, ["country_id"]] = 250 # France
trustworthiness_scores.loc[trustworthiness_scores.country_id == 579, ["country_id"]] = 578 # Norway
trustworthiness_scores.loc[trustworthiness_scores.country_id == 699, ["country_id"]] = 356 # India
trustworthiness_scores.loc[trustworthiness_scores.country_id == 757, ["country_id"]] = 756 # Switzerland
trustworthiness_scores.loc[trustworthiness_scores.country_id == 842, ["country_id"]] = 840 # USA
## 490 is ASIA Others - Not present in Atlas
trustworthiness_scores[~trustworthiness_scores.country_id.isin(atlas_countries.country_id)]["country_id"].unique()

array([886, 975,  58, 866, 588, 230,  97, 280, 532, 490, 590, 459, 717,
       461, 658, 711, 736, 835, 841, 836], dtype=int64)

In [ ]:
#trustworthiness_scores.to_csv("trustworthiness_scores.csv", index=False)

Codes in BACI not matching codes in ATLAS

In [9]:
baci[~baci.exporter.isin(atlas_countries.country_id)]["exporter"].unique()

array([251, 490, 579, 699, 757, 842], dtype=int64)

In [11]:
for file in os.listdir(f"{data_paths['baci']}"):
    print(file)
    _ = pd.read_csv(f"{data_paths['baci']}/" + file, dtype={"k": str})
    if "year" in _.columns:
        continue
    _.rename(columns={"t": "year", "k": "product_id", "i": "exporter", "j": "importer", "v": "export_value", "q": "qty"}, inplace=True)
    _.loc[_.exporter == 251, ["exporter"]] = 250 # France
    _.loc[_.exporter == 579, ["exporter"]] = 578 # Norway
    _.loc[_.exporter == 699, ["exporter"]] = 356 # India
    _.loc[_.exporter == 757, ["exporter"]] = 756 # Switzerland
    _.loc[_.exporter == 842, ["exporter"]] = 840 # USA
    _.loc[_.importer == 251, ["importer"]] = 250 # France
    _.loc[_.importer == 579, ["importer"]] = 578 # Norway
    _.loc[_.importer == 699, ["importer"]] = 356 # India
    _.loc[_.importer == 757, ["importer"]] = 756 # Switzerland
    _.loc[_.importer == 842, ["importer"]] = 840 # USA
    
    print(_.columns)
    print(_[~_.exporter.isin(atlas_countries.country_id)]["exporter"].unique(), _[~_.importer.isin(atlas_countries.country_id)]["importer"].unique())
    #y = input("save?")
    #if y == "y":
    #    _.to_csv("../data/BACI_HS12_V202501/" + file)

BACI_HS12_Y2012_V202501.csv
BACI_HS12_Y2013_V202501.csv
BACI_HS12_Y2014_V202501.csv
BACI_HS12_Y2015_V202501.csv
BACI_HS12_Y2016_V202501.csv
BACI_HS12_Y2017_V202501.csv
BACI_HS12_Y2018_V202501.csv
BACI_HS12_Y2019_V202501.csv
BACI_HS12_Y2020_V202501.csv
BACI_HS12_Y2021_V202501.csv
BACI_HS12_Y2022_V202501.csv
BACI_HS12_Y2023_V202501.csv
country_codes_V202501.csv


AttributeError: 'DataFrame' object has no attribute 'exporter'

Countries in Atlas with no bilateral recorded flows in UNCommtrade

In [ ]:
atlas_countries.head()

,country_id,name_short_en,iso3_code,legacy_location_id
0,533,Aruba,ABW,0
1,4,Afghanistan,AFG,1
2,24,Angola,AGO,2
3,660,Anguilla,AIA,3
4,8,Albania,ALB,4
...,...,...,...,...
247,894,Zambia,ZMB,247
248,716,Zimbabwe,ZWE,248
249,158,Taiwan,TWN,249
250,914,Services Partners,USP,250


In [ ]:
d = {"Country ID": [], "Country Name": [], "Country ISO Code": [], "Reason": []}
for missing in atlas_df[~atlas_df.country_id.isin(trustworthiness_scores.country_id)]["country_id"].unique():
    missing_alpha = atlas_countries.loc[atlas_countries.country_id == missing, "iso3_code"].values[0]
    #print(missing_alpha, missing_alpha in list(reporters.reporterCodeIsoAlpha3))
    d["Country ID"].append(missing)
    d["Country Name"].append(atlas_countries.loc[atlas_countries.country_id == missing, "name_short_en"].values[0])
    d["Country ISO Code"].append(atlas_countries.loc[atlas_countries.country_id == missing, "iso3_code"].values[0])
    
    if missing_alpha in list(reporters.reporterCodeIsoAlpha3):
        d["Reason"].append("No reported data")
    else:
        d["Reason"].append("Country not present in UN Comm Trade dataset")

In [ ]:
#pd.DataFrame(d).sort_values("Country ID").to_latex("test.txt", index=False)